<a href="https://colab.research.google.com/github/ngaihtejames/TubeVault/blob/main/TubeVault.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎥 Smart YouTube Downloader (yt-dlp)

This notebook auto-detects whether the link is a **video** or a **playlist**, and gives you the right quality options automatically — just like YouTube’s quality menu.

✅ Single videos → dropdown with *exact* available qualities  
✅ Playlists → choose a *target quality* (applies to all videos, with graceful fallback)  
✅ Optional cookies.txt upload for login/age-restricted videos  
✅ Downloads to **MyDrive/YT-Downloads**


In [1]:
#@title ▶️ Setup (Run First)
!pip install -U yt-dlp > /dev/null
!apt-get install -y ffmpeg > /dev/null
from google.colab import drive
import os
drive.mount('/content/drive')
os.makedirs('/content/drive/MyDrive/YT-Downloads', exist_ok=True)
print('✅ Setup complete. Files will save to MyDrive/YT-Downloads')

Mounted at /content/drive
✅ Setup complete. Files will save to MyDrive/YT-Downloads


In [3]:
#@title (Optional) Upload cookies.txt
from google.colab import files
uploaded = files.upload()
if 'cookies.txt' in uploaded:
    print('✅ Cookies uploaded.')
else:
    print('ℹ️ No cookies.txt uploaded. Proceeding without.')

Saving cookies.txt to cookies.txt
✅ Cookies uploaded.


In [5]:
#@title 🎬 Enter YouTube Link
yt_url = "https://youtube.com/playlist?list=PLxMSgS0DXn5nODi3UM-SykFVPdeHCxr8p&si=Gj-Gvk8MJpyvrXdD" #@param {type:"string"}

import json, subprocess
if not yt_url.strip():
    raise ValueError("⚠️ Please enter a YouTube link above.")

# Probe metadata
probe = subprocess.run(["yt-dlp", "-J", yt_url, "--flat-playlist"], capture_output=True, text=True)
data = json.loads(probe.stdout)

is_playlist = "entries" in data
print("🔍 Detected:", "Playlist" if is_playlist else "Single Video")

if not is_playlist:
    # Get all available formats for a single video
    probe = subprocess.run(["yt-dlp", "-F", yt_url], capture_output=True, text=True)
    print(probe.stdout)
    print("👆 Above are the available formats. Choose one in the next cell.")
else:
    print("ℹ️ Playlist detected. You’ll select a target quality in the next cell.")

🔍 Detected: Playlist
ℹ️ Playlist detected. You’ll select a target quality in the next cell.


In [6]:
#@title 🎚️ Select Quality & Download
is_playlist = 'list=' in yt_url or 'playlist' in yt_url

if not is_playlist:
    chosen_format = "best" #@param {type:"string"}
    print(f"▶️ Downloading single video in format: {chosen_format}")
    cmd = f"yt-dlp -f {chosen_format} --no-playlist -o '/content/drive/MyDrive/YT-Downloads/%(title)s.%(ext)s'"
else:
    target_quality = "480p" #@param ["144p","240p","360p","480p","720p","1080p","best","audio"]
    print(f"▶️ Downloading playlist with target: {target_quality}")

    if target_quality == "audio":
        cmd = "yt-dlp -f 'bestaudio' -x --audio-format mp3"
    elif target_quality == "best":
        cmd = "yt-dlp -f 'bestvideo+bestaudio/best'"
    else:
        # map resolution to format selector with fallback
        res_map = {
            "144p": "bestvideo[height<=144]+bestaudio/best[height<=144]",
            "240p": "bestvideo[height<=240]+bestaudio/best[height<=240]",
            "360p": "bestvideo[height<=360]+bestaudio/best[height<=360]",
            "480p": "bestvideo[height<=480]+bestaudio/best[height<=480]",
            "720p": "bestvideo[height<=720]+bestaudio/best[height<=720]",
            "1080p": "bestvideo[height<=1080]+bestaudio/best[height<=1080]",
        }
        cmd = f"yt-dlp -f '{res_map[target_quality]}'"

    cmd += f" -o '/content/drive/MyDrive/YT-Downloads/%(title)s.%(ext)s' '{yt_url}'"

if 'cookies.txt' in os.listdir():
    cmd += " --cookies cookies.txt"

print('▶️ Running:', cmd)
!$cmd

▶️ Downloading playlist with target: 480p
▶️ Running: yt-dlp -f 'bestvideo[height<=480]+bestaudio/best[height<=480]' -o '/content/drive/MyDrive/YT-Downloads/%(title)s.%(ext)s' 'https://youtube.com/playlist?list=PLxMSgS0DXn5nODi3UM-SykFVPdeHCxr8p&si=Gj-Gvk8MJpyvrXdD' --cookies cookies.txt
[youtube:tab] Extracting URL: https://youtube.com/playlist?list=PLxMSgS0DXn5nODi3UM-SykFVPdeHCxr8p&si=Gj-Gvk8MJpyvrXdD
[youtube:tab] PLxMSgS0DXn5nODi3UM-SykFVPdeHCxr8p: Downloading webpage
[youtube:tab] PLxMSgS0DXn5nODi3UM-SykFVPdeHCxr8p: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: DiyPerks
[youtube:tab] PLxMSgS0DXn5nODi3UM-SykFVPdeHCxr8p page 1: Downloading API JSON
[youtube:tab] Playlist DiyPerks: Downloading 7 items of 7
[download] Downloading item 1 of 7
[youtube] Extracting URL: https://www.youtube.com/watch?v=W4PHhurAhwc
[youtube] W4PHhurAhwc: Downloading webpage
[youtube] W4PHhurAhwc: Downloading tv client config
[youtube] W4PHhurAhwc: Downloading pla